In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas
import geojson
import binascii
import math

In [2]:
data = pd.read_csv("./occurenzes.csv")
stops = pd.read_json("all.json")["data"]["0"]
data

,junction,direction,request_status
0,300,2,2
1,511,8,3
2,1274,1,0
3,281,1,2
4,1404,2,0
...,...,...,...
5481237,209,1,0
5481238,1406,4,1
5481239,236,2,3
5481240,127,3,0


In [3]:
counted = data.groupby(['junction', 'direction', 'request_status']).size().reset_index(name='count')
counted

,junction,direction,request_status,count
0,0,0,0,17
1,0,0,1,42
2,0,0,2,5
3,0,0,3,2
4,0,1,0,5
...,...,...,...,...
37798,1638,2,3,5
37799,1638,3,0,4
37800,1638,3,1,8
37801,1638,3,2,6


In [4]:
dump = []

# only take into consideration position with more than 300 telegrams
filtered = counted[counted['count'] > 300]

# taking the log so it looks better
filtered['count'].map(math.log)

# normalizing data
filtered['count'] = ((filtered['count'] - min(filtered['count'])) / max(filtered['count']))

# iterating over stops.json
for key, junction in stops.items():
    for stop in junction:
        junction_number = int(key)
        direction = int(stop["direction"])
        request_status = int(stop["request_status"])
        
        # getting the row which matches the stop.json entry
        elem = filtered[
                    (filtered['junction'] == junction_number) &
                    (filtered['direction'] == direction) &
                    (filtered['request_status'] == request_status)
        ]
         
        if len(elem) > 0:
            # generating the color
            color = plt.cm.copper(1 - elem['count'])
            color = list(map(lambda x: int(x * 255), color[0]))
            
            # custom properties for geojson
            properties = {
                "color": ''.join(format(x, '02x') for x in color[0:3])
            }
            
            # add geojson feature
            dump.append(geojson.Feature(geometry=geojson.Point((stop["lon"], stop["lat"])), properties=properties))
features = geojson.FeatureCollection(dump)
features

301


/run/user/1000/ipykernel_1213163/4043354558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['count'] = ((filtered['count'] - min(filtered['count'])) / max(filtered['count']))


{"features": [{"geometry": {"coordinates": [13.783373, 51.067084], "type": "Point"}, "properties": {"color": "ffc47c"}, "type": "Feature"}, {"geometry": {"coordinates": [13.783013, 51.067012], "type": "Point"}, "properties": {"color": "ffbb77"}, "type": "Feature"}, {"geometry": {"coordinates": [13.784868, 51.067001], "type": "Point"}, "properties": {"color": "ffc37c"}, "type": "Feature"}, {"geometry": {"coordinates": [13.785432, 51.066972], "type": "Point"}, "properties": {"color": "ffb573"}, "type": "Feature"}, {"geometry": {"coordinates": [13.788509, 51.067183], "type": "Point"}, "properties": {"color": "ffc47c"}, "type": "Feature"}, {"geometry": {"coordinates": [13.785366, 51.067074], "type": "Point"}, "properties": {"color": "ffc37c"}, "type": "Feature"}, {"geometry": {"coordinates": [13.785005, 51.066966], "type": "Point"}, "properties": {"color": "ffae6f"}, "type": "Feature"}, {"geometry": {"coordinates": [13.732693, 51.077298], "type": "Point"}, "properties": {"color": "ffa166"}